In [1]:
#imports 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
# python -m spacy download en_core_web_sm
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# pip install spacy==2.2.4

In [5]:
import en_core_web_lg
en_core_web_lg.load()

In [6]:
#loading data
Email = pd.read_csv('email_headers.csv', encoding='cp1252')
# Employee = pd.read_excel('EmployeeRecords.xlsx')

Email = pd.read_csv('email_headers.csv', encoding='cp1252')
df_text = Email[['From', 'Subject']].drop_duplicates('Subject')
df_text['Subject'].to_csv('subject.csv', index=False)
# df_text = pd.read_excel('Subject.xlsx')
df_text['Subject'] = df_text['Subject'].apply(lambda x: x[4:] if x.startswith('Re: ') or x.startswith('RE: ') else x )
df_text = df_text.drop_duplicates(subset = ['Subject'], keep='first' )
nlp=en_core_web_lg.load() #spacy.load("en_core_web_lg")
df_text['token'] = df_text['Subject'].apply(lambda x: nlp(x))
df_text


,From,Subject,token
0,Sven.Flecha@gastech.com.kronos,GT-SeismicProcessorPro Bug Report,"(GT, -, SeismicProcessorPro, Bug, Report)"
1,Kanon.Herrero@gastech.com.kronos,Inspection request for site,"(Inspection, request, for, site)"
2,Bertrand.Ovan@gastech.com.kronos,New refueling policies - Effective February 1,"(New, refueling, policies, -, Effective, Febru..."
3,Valeria.Morlun@gastech.com.kronos,Route suggestion for next shift,"(Route, suggestion, for, next, shift)"
4,Mat.Bramar@gastech.com.kronos,Upcoming birthdays,"(Upcoming, birthdays)"
...,...,...,...
886,Ada.Campo-Corrente@gastech.com.kronos,Deadline changed?,"(Deadline, changed, ?)"
919,Mat.Bramar@gastech.com.kronos,Overtime policy reminder,"(Overtime, policy, reminder)"
929,Mat.Bramar@gastech.com.kronos,Registration open: retirement planning seminar,"(Registration, open, :, retirement, planning, ..."
1042,Marin.Onda@gastech.com.kronos,NO DECAF - HA HA,"(NO, DECAF, -, HA, HA)"


In [7]:
# !pip uninstall gensim

In [8]:
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags, STOPWORDS


In [9]:
df_text.columns

Index(['From', 'Subject', 'token'], dtype='object')

In [10]:
df_class = pd.DataFrame(columns = ['Class', 'Description'], data= [['Spam', 'Spam'],['Social', 'Anniversary Birthday Event Social Night Evening'],['Work', 'Report Team Data Meeting Inspection Visit IPO Protocol Supplies GasTech Files'], [ 'Change/schedule', 'Change Schedule Appointments Meeting Deadline New Shift Announcement'],['Weird', 'Weird Plants text and drive'],['Other', 'Other No Hurry'],['Undefined', 'Undefined']])
df_class['token'] = df_class['Description'].apply(lambda x: nlp(x))
df_class

classes = df_class['Class']
for index, row in df_class.iterrows():
    row
    x = df_text['token'].apply(lambda x: x.similarity(row['token']))
    print(row['Class'])


def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=4, max_len=15) #originally len 2 and 15



df_text['tagged_docs'] = df_text.apply(lambda row: TaggedDocument(tokenize(row['Subject']), [str(row.name)]), axis=1)


Spam
Social
Work
Change/schedule
Weird
Other
Undefined


In [11]:
df_text

,From,Subject,token,tagged_docs
0,Sven.Flecha@gastech.com.kronos,GT-SeismicProcessorPro Bug Report,"(GT, -, SeismicProcessorPro, Bug, Report)","([report], [0])"
1,Kanon.Herrero@gastech.com.kronos,Inspection request for site,"(Inspection, request, for, site)","([inspection, request, site], [1])"
2,Bertrand.Ovan@gastech.com.kronos,New refueling policies - Effective February 1,"(New, refueling, policies, -, Effective, Febru...","([refueling, policies, effective, february], [2])"
3,Valeria.Morlun@gastech.com.kronos,Route suggestion for next shift,"(Route, suggestion, for, next, shift)","([route, suggestion, next, shift], [3])"
4,Mat.Bramar@gastech.com.kronos,Upcoming birthdays,"(Upcoming, birthdays)","([upcoming, birthdays], [4])"
...,...,...,...,...
886,Ada.Campo-Corrente@gastech.com.kronos,Deadline changed?,"(Deadline, changed, ?)","([deadline, changed], [886])"
919,Mat.Bramar@gastech.com.kronos,Overtime policy reminder,"(Overtime, policy, reminder)","([overtime, policy, reminder], [919])"
929,Mat.Bramar@gastech.com.kronos,Registration open: retirement planning seminar,"(Registration, open, :, retirement, planning, ...","([registration, open, retirement, planning, se..."
1042,Marin.Onda@gastech.com.kronos,NO DECAF - HA HA,"(NO, DECAF, -, HA, HA)","([decaf], [1042])"


In [12]:
classes = df_class['Class']
for index, row in df_class.iterrows():
    row
    df_text[str(row['Class'])] = df_text['token'].apply(lambda x: x.similarity(row['token']))
    # print(row['Class'])

In [13]:
df_text

,From,Subject,token,tagged_docs,Spam,Social,Work,Change/schedule,Weird,Other,Undefined
0,Sven.Flecha@gastech.com.kronos,GT-SeismicProcessorPro Bug Report,"(GT, -, SeismicProcessorPro, Bug, Report)","([report], [0])",0.355546,0.323271,0.546655,0.434195,0.485078,0.394106,0.274350
1,Kanon.Herrero@gastech.com.kronos,Inspection request for site,"(Inspection, request, for, site)","([inspection, request, site], [1])",0.349533,0.434018,0.782635,0.612628,0.520210,0.563195,0.116503
2,Bertrand.Ovan@gastech.com.kronos,New refueling policies - Effective February 1,"(New, refueling, policies, -, Effective, Febru...","([refueling, policies, effective, february], [2])",0.255670,0.456037,0.590558,0.648555,0.524386,0.484063,0.160460
3,Valeria.Morlun@gastech.com.kronos,Route suggestion for next shift,"(Route, suggestion, for, next, shift)","([route, suggestion, next, shift], [3])",0.266890,0.599843,0.619576,0.761161,0.673466,0.687917,0.148904
4,Mat.Bramar@gastech.com.kronos,Upcoming birthdays,"(Upcoming, birthdays)","([upcoming, birthdays], [4])",0.109208,0.672504,0.303903,0.529019,0.292671,0.347388,0.012583
...,...,...,...,...,...,...,...,...,...,...,...
886,Ada.Campo-Corrente@gastech.com.kronos,Deadline changed?,"(Deadline, changed, ?)","([deadline, changed], [886])",0.281788,0.489065,0.509307,0.745960,0.523024,0.586996,0.191004
919,Mat.Bramar@gastech.com.kronos,Overtime policy reminder,"(Overtime, policy, reminder)","([overtime, policy, reminder], [919])",0.285913,0.485361,0.523834,0.685068,0.458422,0.521070,0.163929
929,Mat.Bramar@gastech.com.kronos,Registration open: retirement planning seminar,"(Registration, open, :, retirement, planning, ...","([registration, open, retirement, planning, se...",0.201248,0.602333,0.682553,0.753981,0.465848,0.512816,0.101617
1042,Marin.Onda@gastech.com.kronos,NO DECAF - HA HA,"(NO, DECAF, -, HA, HA)","([decaf], [1042])",0.255405,0.271285,0.176883,0.230800,0.383968,0.507482,0.079079


In [14]:
x

0       0.274350
1       0.116503
2       0.160460
3       0.148904
4       0.012583
          ...   
886     0.191004
919     0.163929
929     0.101617
1042    0.079079
1098    0.042775
Name: token, Length: 157, dtype: float64

In [15]:
df_text['class'] = df_text[['Spam', 'Social', 'Work', 'Change/schedule', 'Weird', 'Other', 'Undefined']].idxmax(axis=1)


In [16]:
df_text.columns

Index(['From', 'Subject', 'token', 'tagged_docs', 'Spam', 'Social', 'Work',
       'Change/schedule', 'Weird', 'Other', 'Undefined', 'class'],
      dtype='object')

In [17]:
df_text.head(5)

,From,Subject,token,tagged_docs,Spam,Social,Work,Change/schedule,Weird,Other,Undefined,class
0,Sven.Flecha@gastech.com.kronos,GT-SeismicProcessorPro Bug Report,"(GT, -, SeismicProcessorPro, Bug, Report)","([report], [0])",0.355546,0.323271,0.546655,0.434195,0.485078,0.394106,0.274350,Work
1,Kanon.Herrero@gastech.com.kronos,Inspection request for site,"(Inspection, request, for, site)","([inspection, request, site], [1])",0.349533,0.434018,0.782635,0.612628,0.520210,0.563195,0.116503,Work
2,Bertrand.Ovan@gastech.com.kronos,New refueling policies - Effective February 1,"(New, refueling, policies, -, Effective, Febru...","([refueling, policies, effective, february], [2])",0.255670,0.456037,0.590558,0.648555,0.524386,0.484063,0.160460,Change/schedule
3,Valeria.Morlun@gastech.com.kronos,Route suggestion for next shift,"(Route, suggestion, for, next, shift)","([route, suggestion, next, shift], [3])",0.266890,0.599843,0.619576,0.761161,0.673466,0.687917,0.148904,Change/schedule
4,Mat.Bramar@gastech.com.kronos,Upcoming birthdays,"(Upcoming, birthdays)","([upcoming, birthdays], [4])",0.109208,0.672504,0.303903,0.529019,0.292671,0.347388,0.012583,Social


In [18]:
df_text[['From', 'Subject', 'class']].to_csv('email_classification.csv')

In [19]:
# LargeEmail = LargeEmail.merge(df_text, on='Subject')